**Lecture 4 - Dependency Parsing**

**Linguistic Structure**
- 2 views:
    - 1. **phrase structure** = constituency = context-free grammars (CFGs)
        - organizes words into nested constituents
    - 2. **dependency structure**
        - shows which words depend on (modify, attach to, or arguments of) which other words

- linear stream of words
- problems
    - prepositional phrase attachment ambiguity
    - coordination scope ambiguity
    - adjectival/adverbial modifier ambiguity
    - verb phrase attachment ambiguity
- solution: dependency paths help extract semantic interpretation

**Dependency Grammar and Dependency Structure**

- dependency syntax suggests that syntactic structure consists of relations between lexical items, normally binary asymmetric relations (arrows) called dependencies
    - arrows generally point from head to dependent
- usually dependencies form a tree
- usually add a fake ROOT so every word is a dependent of precisely 1 other node
    - Ex. "ROOT Dsicussion of the outstanding issues was completed."

- the rise of annotated data & Universal Dependencies treebanks
    - a lot slower and less useful than writing a grammar by hand
    - but gives us:
        - reusability of the labor
        - broad coverage, not just a few intuitions
        - frequencies and distributional information
        - a way to evaluate NLP systems
        

**Dependency Conditioning Preferences**

- straightforward sources of information for dependency parsing:
    - 1. **bilexical affinities** - the dependency is plausible
    - 2. **dependency distance** - most dependencies are between nearby words
    - 3. **intervening material** - dependencies rarely span intervening verbs or punctuation
    - 4. **valency of heads** - how many dependents on which side are usual for a head?

**Dependency Parsing**

- a sentence is parsed by choosing for each word what other word (including ROOT) it is a dependent of

- usually some constraints:
    - only one word is a dependent of ROOT
    - don't want cycles A --> B, B --> A

- makes the dependencies a tree
- final issue: whether arrows can cross (be non-projective) or not

- **projective parse** - there are no crossing dependency arcs when the words are laid out in their linear order, with all arcs above the words
    - dependencies corresponding to a CFG tree must be projective (by forming dependencies by taking 1 child of each category as head)
    - most syntactic structure is projective, but dependency theory normally does allow non-projective structures to account for displaced constituents

- methods:
    - 1. dynamic programming
    - 2. graph algorithms
    - 3. contraint satisfaction
    - 4. transition-based parsing or deterministic dependency parsing

- **transition-based parsing** (greedy)
    - simple form of a greedy discriminative dependency parser
    - parser does a sequence of bottom-up actions
        - like shift or reduce
    - parser has:
        - a stack _sigmoid_, written with top to the right, which starts with ROOT symbol
        - a buffer _Beta_, written with top to the left, which starts with the input sentence
        - a set of dependency arcs A, which starts off empty
        - a set of actions (arc-building actions)
    - ex. "I ate fish" (arc-standard transition-based parser)
        - stack: [root], buffer: [I ate fish]
        - shift --> stack: [root I ate] --> [root ate] = left arc, dependency = nsubj(ate --> I)
        - shift --> stack: [root ate][fish] --> [root ate fish]
        - stack: [root ate fish] --> [root ate], right arc, dependency = obj(ate --> fish)
        - stack: [root ate] --> [root], right arc, dependency = root(root --> ate)
        - finish

- we have left to explain how we choose the next action --> machine learning
    - each action is predicted by a discriminative classifier over each legal move
        - max of 3 untyped choices
        - features: top of stack word, POS; first in buffer word, POS; etc.
    - sequence of transition predictions / ML operations
    - there is no search (in the simplest form)
        - but can do a beam search (slower but better)
    - accuracy a bit below state of the art in dependency parsin but VERY fast linear time parsin, which high accuracy - great for parsing the web

- evaluating dependency parsers via (unlabeled or labeled) dependency accuracy


- why do we gain from a neural dependency parser?
    - problem: categorical (indicator) features are sparse, incomplete, expensive to compute
    - neural approach - learn a dense and compacy feature representation
    - unlabeled attachement score (UAS) = head
    - labeled attachement score (LAS) = head and label

    - we can represent words as a d-dimensional dense vector (word embedding)
        - similar words are expected to have close vectors
    - part-of-speech tags (POS) and dependency labels are also represented as d-dimensional vectors
        - smaller discrete sets also exhibit many semantical similarities
            - NNS (plural noun) should be close to NN (singular noun)
            - nummod (numerical modifier) shoudl be close to amod (adjective modifier)

    - extract a set of tokens based on the stack/buffer positions
    - concatenation of the vector representations --> neural representation of a configuration

- **Neural Dependency Parser Model Architecture**
    - a simple feed-forward neural network multi-class classifier
    - operations: shift, left arc, right arc

- deep learning classifiers are non-linear classifiers

- **Graph-based dependency parsers**
    - compute a score for every possible dependency for each word (ex. picking the head for "big")
    - requires good contextual representations of each word token
    - repeat the same process for each other word - find the best parse (MST algorithm - minimum spanning tree)